# Encontro 13: Medidas de Centralidade

Importando a biblioteca:

In [1]:
import sys
sys.path.append('..')

from random import choice
from itertools import permutations

import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import socnet as sn

Configurando a biblioteca:

In [2]:
sn.node_size = 10
sn.node_color = (255, 255, 255)

sn.edge_width = 1
sn.edge_color = (192, 192, 192)

sn.node_label_position = 'top center'

Carregando rede de casamentos entre famílias de Florença durante a Renascença.

J. F. Padgett e C. K. Ansell. *Robust action and the rise of the Medici, 1400–1434.* American Journal of
Sociology 98, págs. 1259-1319, 1993.

In [3]:
g = sn.load_graph('Renaissance.gml', has_pos=True)

sn.show_graph(g, nlab=True)

Função que registra, em cada nó, seus sucessores em geodésicas de $s$ a $t$.

In [4]:
def set_geodesic_successors(g, s, t):
    for n in g.nodes:
        g.nodes[n]['geodesic_successors'] = set()

    for p in nx.all_shortest_paths(g, s, t):
        for i in range(len(p) - 1):
            g.nodes[p[i]]['geodesic_successors'].add(p[i + 1])

Funções que representam uma escolha aleatória de sucessor para diferentes tipos de trajetórias.

In [5]:
# Pense que o atributo 'passages' abaixo indica quantas
# vezes um fluxo já passou por um nó ou por uma aresta.

def random_geodesic_successor(g, n):
    return choice([m for m in g.nodes[n]['geodesic_successors']])

def random_path_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.nodes[m]['passages'] == 0])

def random_trail_successor(g, n):
    return choice([m for m in g.neighbors(n) if g.edges[n, m]['passages'] == 0])

def random_walk_successor(g, n):
    return choice([m for m in g.neighbors(n)])

Função que faz uma simulação de fluxo de $s$ a $t$, que pode ou não ser bem-sucedida.

In [6]:
def simulate_single_flow(g, s, t,func,dono_insumo):
    functions = [random_geodesic_successor,random_path_successor,random_trail_successor, random_walk_successor ]
    # Inicializa o atributo 'passages' de cada nó.
    for n in g.nodes:
        g.nodes[n]['passages'] = 0
    g.nodes[s]['passages'] = 1

    # Inicializa o atributo 'passages' de cada aresta.
    for n, m in g.edges:
        g.edges[n, m]['passages'] = 0

    # Inicializa s como o único dono do insumo.
    for n in g.nodes:
        g.nodes[n]['owner'] = False
    g.nodes[s]['owner'] = True

    # Simula o fluxo, contando o número total de passos.

    steps = 0

    while True:
        # O conjunto reached representa todos os nós
        # que o fluxo consegue alcançar no passo atual.
        reached = set()

        # Verifica cada um dos donos atuais do insumo.

        owners = [n for n in g.nodes if g.nodes[n]['owner']]

        for n in owners:
            
            if(not dono_insumo):
                # Deixa de ser dono do insumo.
                g.nodes[n]['owner'] = False
                
                
            # Escolhe aleatoriamente um dos sucessores.
            try:
                m = functions[func-1](g, n)
            except IndexError:
                continue

            

            # Incrementa o atributo 'passages' do nó.
            g.nodes[m]['passages'] += 1

            # Incrementa o atributo 'passages' da aresta.
            g.edges[n, m]['passages'] += 1

            # Registra que consegue alcançar esse nó.
            reached.add(m)

        # Todo nó alcançado passa a ser dono do insumo.

        for n in reached:
            g.nodes[n]['owner'] = True

        # Isso conclui o passo atual da simulação.
        steps += 1

        # Se o passo alcançou t, chegamos ao fim da simulação.
        # Ela foi bem-sucedida: devolvemos o número de passos.
        if t in reached:
            return steps

        # Se o passo não alcançou ninguém, chegamos ao fim da
        # simulação. Ela não foi bem-sucedida: devolvemos -1.
        if not reached:
            return -1

Função que faz simulações de fluxo de $s$ a $t$ até uma ser bem-sucedida.

In [7]:
def simulate_successful_flow(g, s, t,func,dono_insumo):
    set_geodesic_successors(g, s, t)

    while True:
        steps = simulate_single_flow(g, s, t,func,dono_insumo)

        if steps != -1:
            return steps

Função que faz simulações de fluxo para todo $s$ e $t$ possíveis, e tira disso um *closeness simulado* e um *betweenness simulado*.

In [8]:
def simulate_all_flows(g,func,dono_insumo):
    for n in g.nodes:
        g.nodes[n]['closeness'] = 0
        g.nodes[n]['betweenness'] = 0

    for s, t in permutations(g.nodes, 2):
        steps = simulate_successful_flow(g, s, t,func,dono_insumo)

        g.nodes[s]['closeness'] += steps
        for n in g.nodes:
            if n != s and n != t:
                g.nodes[n]['betweenness'] += g.nodes[n]['passages']

    # Normalizações necessárias para comparar com os
    # resultados analíticos. Não é preciso entender.
    for n in g.nodes:
        g.nodes[n]['closeness'] = (g.number_of_nodes() - 1) / g.nodes[n]['closeness']
        g.nodes[n]['betweenness'] /= (g.number_of_nodes() - 1) * (g.number_of_nodes() - 2)

Média de *closeness simulado* e *betweenness simulado* para muitas repetições da simulação acima.

In [9]:
def run(g,func,dono_insumo):
    TIMES = 100


    for n in g.nodes:
        g.nodes[n]['mean_closeness'] = 0
        g.nodes[n]['mean_betweenness'] = 0

    for _ in range(TIMES):
        simulate_all_flows(g,func,dono_insumo)

        for n in g.nodes:
            g.nodes[n]['mean_closeness'] += g.nodes[n]['closeness']
            g.nodes[n]['mean_betweenness'] += g.nodes[n]['betweenness']

    for n in g.nodes:
        g.nodes[n]['mean_closeness'] /= TIMES
        g.nodes[n]['mean_betweenness'] /= TIMES

    return pd.DataFrame({
        'família': [g.nodes[n]['label'] for n in g.nodes],
        'closeness simulado': [g.nodes[n]['mean_closeness'] for n in g.nodes],
        'closeness analítico': [cc[n] for n in g.nodes],
        'betweenness simulado': [g.nodes[n]['mean_betweenness'] for n in g.nodes],
        'betweenness analítico': [bc[n] for n in g.nodes],
    })

Cálculo de *closeness* e *betweenness* a partir das funções prontas da NetworkX, para comparação.

Construção de data frame só para comparar mais facilmente.

In [10]:
cc = nx.closeness_centrality(g)

bc = nx.betweenness_centrality(g)

In [11]:
geo_duplic = run(g,1, True)
geo_duplic.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
6,medici,0.560000,0.560000,1.390714,0.521978
2,albizzi,0.482759,0.482759,0.570275,0.212454
9,ridolfi,0.482759,0.482759,0.251648,0.086081
7,tornabuon,0.482759,0.482759,0.245385,0.091575
3,guadagni,0.466667,0.466667,0.687692,0.260073
13,barbadori,0.437500,0.437500,0.317198,0.115385
11,strozzi,0.424242,0.424242,0.207143,0.075092
8,bischeri,0.400000,0.400000,0.311648,0.120879
5,salviati,0.388889,0.388889,0.406593,0.142857
14,castellan,0.388889,0.388889,0.230879,0.087912


In [12]:
close_geo_duplic = geo_duplic.iloc[:, 1].values
bet_geo_duplic = geo_duplic.iloc[:, 3].values
geo_duplic.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.411834,0.411834,0.311634,0.115751
std,0.074293,0.074293,0.364629,0.136863
min,0.285714,0.285714,0.000000,0.000000
25%,0.359211,0.359211,0.027665,0.010989
50%,0.400000,0.400000,0.245385,0.087912
75%,0.474713,0.474713,0.361896,0.131868
max,0.560000,0.560000,1.390714,0.521978


In [13]:
geo_transf = run(g,1, False)
geo_transf.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
6,medici,0.560000,0.560000,0.521484,0.521978
2,albizzi,0.482759,0.482759,0.210879,0.212454
9,ridolfi,0.482759,0.482759,0.089835,0.086081
7,tornabuon,0.482759,0.482759,0.089670,0.091575
3,guadagni,0.466667,0.466667,0.256813,0.260073
13,barbadori,0.437500,0.437500,0.115110,0.115385
11,strozzi,0.424242,0.424242,0.079176,0.075092
8,bischeri,0.400000,0.400000,0.120549,0.120879
5,salviati,0.388889,0.388889,0.142857,0.142857
14,castellan,0.388889,0.388889,0.088242,0.087912


In [14]:
close_geo_transf = geo_transf.iloc[:, 1].values
bet_geo_transf = geo_transf.iloc[:, 3].values
geo_transf.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.411834,0.411834,0.115751,0.115751
std,0.074293,0.074293,0.136335,0.136863
min,0.285714,0.285714,0.000000,0.000000
25%,0.359211,0.359211,0.010824,0.010989
50%,0.400000,0.400000,0.089670,0.087912
75%,0.474713,0.474713,0.131703,0.131868
max,0.560000,0.560000,0.521484,0.521978


In [15]:
path_duplic = run(g, 2, True)
path_duplic.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
7,tornabuon,0.278030,0.482759,0.681099,0.091575
9,ridolfi,0.275726,0.482759,0.702088,0.086081
6,medici,0.265837,0.560000,0.820495,0.521978
3,guadagni,0.265041,0.466667,0.743901,0.260073
13,barbadori,0.263984,0.437500,0.580879,0.115385
11,strozzi,0.263603,0.424242,0.680440,0.075092
8,bischeri,0.262429,0.400000,0.650769,0.120879
2,albizzi,0.254752,0.482759,0.616978,0.212454
14,castellan,0.253489,0.388889,0.617637,0.087912
12,peruzzi,0.253409,0.350000,0.526923,0.021978


In [16]:
close_path_duplic = path_duplic.iloc[:, 1].values
bet_path_duplic = path_duplic.iloc[:, 3].values
path_duplic.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.250969,0.411834,0.567934,0.115751
std,0.020911,0.074293,0.157473,0.136863
min,0.203899,0.285714,0.264560,0.000000
25%,0.235105,0.359211,0.421429,0.010989
50%,0.254752,0.400000,0.616978,0.087912
75%,0.264513,0.474713,0.680769,0.131868
max,0.278030,0.560000,0.820495,0.521978


In [17]:
path_transf = run(g,2, False)
path_transf.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
6,medici,0.305926,0.560000,0.629066,0.521978
3,guadagni,0.269255,0.466667,0.402747,0.260073
7,tornabuon,0.261050,0.482759,0.334780,0.091575
9,ridolfi,0.258842,0.482759,0.364341,0.086081
2,albizzi,0.257231,0.482759,0.228736,0.212454
14,castellan,0.253950,0.388889,0.383791,0.087912
8,bischeri,0.246580,0.400000,0.365824,0.120879
13,barbadori,0.242199,0.437500,0.240769,0.115385
5,salviati,0.242110,0.388889,0.142857,0.142857
10,acciaiuol,0.238060,0.368421,0.000000,0.000000


In [18]:
close_path_transf = path_transf.iloc[:, 1].values
bet_path_transf = path_transf.iloc[:, 3].values
path_transf.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.243547,0.411834,0.247593,0.115751
std,0.027027,0.074293,0.188196,0.136863
min,0.198528,0.285714,0.000000,0.000000
25%,0.229726,0.359211,0.071429,0.010989
50%,0.242199,0.400000,0.247253,0.087912
75%,0.258037,0.474713,0.369780,0.131868
max,0.305926,0.560000,0.629066,0.521978


In [19]:
trail_duplic = run(g,3, True)
trail_duplic.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
7,tornabuon,0.261941,0.482759,0.923791,0.091575
9,ridolfi,0.261346,0.482759,0.929780,0.086081
6,medici,0.256561,0.560000,1.253901,0.521978
3,guadagni,0.253668,0.466667,0.900934,0.260073
13,barbadori,0.253498,0.437500,0.602637,0.115385
8,bischeri,0.249716,0.400000,0.780110,0.120879
11,strozzi,0.249646,0.424242,0.883132,0.075092
2,albizzi,0.248451,0.482759,0.688242,0.212454
14,castellan,0.242129,0.388889,0.827692,0.087912
12,peruzzi,0.238096,0.350000,0.592747,0.021978


In [20]:
close_trail_duplic = trail_duplic.iloc[:, 1].values
bet_trail_duplic = trail_duplic.iloc[:, 3].values
trail_duplic.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.241117,0.411834,0.680381,0.115751
std,0.017490,0.074293,0.280204,0.136863
min,0.200382,0.285714,0.262473,0.000000
25%,0.227341,0.359211,0.410330,0.010989
50%,0.248451,0.400000,0.688242,0.087912
75%,0.253583,0.474713,0.892033,0.131868
max,0.261941,0.560000,1.253901,0.521978


In [21]:
trail_transf = run(g,3, False)
trail_transf.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
6,medici,0.291128,0.560000,0.701923,0.521978
2,albizzi,0.276296,0.482759,0.267912,0.212454
13,barbadori,0.259748,0.437500,0.269945,0.115385
3,guadagni,0.257665,0.466667,0.410110,0.260073
7,tornabuon,0.246191,0.482759,0.314505,0.091575
9,ridolfi,0.243966,0.482759,0.332253,0.086081
12,peruzzi,0.240704,0.350000,0.226923,0.021978
5,salviati,0.238170,0.388889,0.142857,0.142857
11,strozzi,0.233742,0.424242,0.327527,0.075092
8,bischeri,0.229909,0.400000,0.341703,0.120879


In [22]:
close_trail_transf = trail_transf.iloc[:, 1].values
bet_trail_transf = trail_transf.iloc[:, 3].values
trail_transf.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.239803,0.411834,0.247476,0.115751
std,0.024669,0.074293,0.195274,0.136863
min,0.196370,0.285714,0.000000,0.000000
25%,0.227567,0.359211,0.071429,0.010989
50%,0.238170,0.400000,0.269945,0.087912
75%,0.251928,0.474713,0.336978,0.131868
max,0.291128,0.560000,0.701923,0.521978


In [23]:
walk_duplic = run(g,4, True)
walk_duplic.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
6,medici,0.173774,0.560000,6.930824,0.521978
7,tornabuon,0.168356,0.482759,2.396593,0.091575
9,ridolfi,0.166838,0.482759,2.528956,0.086081
13,barbadori,0.162888,0.437500,1.473022,0.115385
3,guadagni,0.162499,0.466667,4.509670,0.260073
2,albizzi,0.160570,0.482759,3.040110,0.212454
11,strozzi,0.157669,0.424242,2.707473,0.075092
8,bischeri,0.153606,0.400000,2.760604,0.120879
14,castellan,0.152636,0.388889,3.110275,0.087912
10,acciaiuol,0.152590,0.368421,0.580220,0.000000


In [24]:
close_walk_duplic = walk_duplic.iloc[:, 1].values
bet_walk_duplic = walk_duplic.iloc[:, 3].values
walk_duplic.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.154848,0.411834,2.386201,0.115751
std,0.011548,0.074293,1.675391,0.136863
min,0.129184,0.285714,0.580220,0.000000
25%,0.145917,0.359211,1.130467,0.010989
50%,0.153606,0.400000,2.396593,0.087912
75%,0.162693,0.474713,2.900357,0.131868
max,0.173774,0.560000,6.930824,0.521978


In [25]:
walk_transf = run(g,4, False)
walk_transf.sort_values(by=['closeness simulado','betweenness simulado'], ascending=False)

,família,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
5,salviati,0.040369,0.388889,1.536758,0.142857
4,pazzi,0.039047,0.285714,0.687582,0.000000
12,peruzzi,0.038499,0.350000,1.589231,0.021978
0,ginori,0.037620,0.333333,0.770330,0.000000
10,acciaiuol,0.036487,0.368421,0.760989,0.000000
13,barbadori,0.036468,0.437500,1.632582,0.115385
1,lambertes,0.036308,0.325581,0.773407,0.000000
8,bischeri,0.036138,0.400000,2.481703,0.120879
2,albizzi,0.036038,0.482759,2.510055,0.212454
7,tornabuon,0.035388,0.482759,2.507692,0.091575


In [26]:
close_walk_transf = walk_transf.iloc[:, 1].values
bet_walk_transf = walk_transf.iloc[:, 3].values
walk_transf.describe()

,closeness simulado,closeness analítico,betweenness simulado,betweenness analítico
count,15.000000,15.000000,15.000000,15.000000
mean,0.036288,0.411834,2.071648,0.115751
std,0.001894,0.074293,1.174852,0.136863
min,0.034060,0.285714,0.687582,0.000000
25%,0.034689,0.359211,1.155082,0.010989
50%,0.036138,0.400000,2.438462,0.087912
75%,0.037053,0.474713,2.505055,0.131868
max,0.040369,0.560000,5.056044,0.521978


E agora, vamos pensar um pouco...

* Onde você precisa mudar o código para usar uma *trajetória* que não seja a *geodésica*? (caminho, trilha, passeio)

* Onde você precisa mudar o código para usar uma *difusão* que não seja a *transferência*? (duplicação)

Considere então a seguinte **hipótese**:

>Quando consideramos outros tipos de trajetória e outros tipos de difusão, os nós com maior *closeness simulado* e *betweenness simulado* não são necessariamente os nós com maior *closeness* e *betweenness* segundo as fórmulas clássicas. (que correspondem ao uso de geodésica e transferência na simulação)

Queremos:

1. Operacionalização e teste dessas hipótese. (Objetivo 3)
2. Interpretação dos resultados na linguagem de Análise de Redes Sociais (Objetivo 4)

Um *feedback* da atividade sobre *coreness no Jazz* será dado em breve, para vocês terem uma melhor referência do item 2.

In [27]:
from scipy import stats
pd.options.display.float_format = '{:.10f}'.format

In [28]:
closeness_array = [close_geo_duplic, 
close_path_duplic, close_path_transf,
close_trail_duplic, close_trail_transf,
close_walk_duplic, close_walk_transf]

betweenness_array = [bet_geo_duplic,
bet_path_duplic, bet_path_transf,
bet_trail_duplic, bet_trail_transf,
bet_walk_duplic, bet_walk_transf]

closeness_tstatistic = []
closeness_pvalue = []
betweenness_tstatistic = []
betweenness_pvalue = []

for i in range(len(closeness_array)):
    test = stats.ttest_rel(closeness_array[i], close_geo_transf)
    closeness_tstatistic.append(test[0])
    closeness_pvalue.append(test[1])

for i in range(len(betweenness_array)):
    test = stats.ttest_rel(betweenness_array[i], bet_geo_transf)
    betweenness_tstatistic.append(test[0])
    betweenness_pvalue.append(test[1])
    
types_array = ["geo_duplic",
"path_duplic", "path_transf",
"trail_duplic", "trail_transf",
"walk_duplic", "walk_transf"]
    
t_tests_df = pd.DataFrame({
        'type': [e for e in types_array],
        'closeness tstatistic': [e for e in closeness_tstatistic],
        'closeness pvalue': [e for e in closeness_pvalue],
        'betweenness tstatistic': [e for e in betweenness_tstatistic],
        'betweenness pvalue': [e for e in betweenness_pvalue]
    })
t_tests_df

,type,closeness tstatistic,closeness pvalue,betweenness tstatistic,betweenness pvalue
0,geo_duplic,nan,nan,3.3222654061,0.0050342445
1,path_duplic,-10.5792814778,0.0000000463,15.5675537812,0.0000000003
2,path_transf,-13.0864013178,0.0000000031,4.1807112033,0.0009245877
3,trail_duplic,-11.0114660352,0.0000000280,10.8380471286,0.0000000342
4,trail_transf,-12.5156219879,0.0000000054,4.6552869871,0.0003713412
5,walk_duplic,-15.6869415040,0.0000000003,5.6796793006,0.0000568575
6,walk_transf,-19.2630467294,0.0000000000,7.1954574733,0.0000045973


In [29]:
t_tests_df.sort_values(by=['closeness pvalue','betweenness pvalue'], ascending=False)

,type,closeness tstatistic,closeness pvalue,betweenness tstatistic,betweenness pvalue
1,path_duplic,-10.5792814778,0.0000000463,15.5675537812,0.0000000003
3,trail_duplic,-11.0114660352,0.0000000280,10.8380471286,0.0000000342
4,trail_transf,-12.5156219879,0.0000000054,4.6552869871,0.0003713412
2,path_transf,-13.0864013178,0.0000000031,4.1807112033,0.0009245877
5,walk_duplic,-15.6869415040,0.0000000003,5.6796793006,0.0000568575
6,walk_transf,-19.2630467294,0.0000000000,7.1954574733,0.0000045973
0,geo_duplic,nan,nan,3.3222654061,0.0050342445


In [30]:
t_tests_df.describe()

,closeness tstatistic,closeness pvalue,betweenness tstatistic,betweenness pvalue
count,6.0000000000,6.0000000000,7.0000000000,7.0000000000
mean,-13.6904598420,0.0000000138,7.3484287543,0.0009130947
std,3.2754002186,0.0000000191,4.3976701393,0.0018485776
min,-19.2630467294,0.0000000000,3.3222654061,0.0000000003
25%,-15.0368064575,0.0000000010,4.4179990952,0.0000023157
50%,-12.8010116528,0.0000000042,5.6796793006,0.0000568575
75%,-11.3875050234,0.0000000223,9.0167523010,0.0006479645
max,-10.5792814778,0.0000000463,15.5675537812,0.0050342445
